# Scikit-learn 내장 데이터셋: load_digits 분석 및 분류 모델 프로젝트

이 노트북은 Scikit-learn에서 제공하는 손글씨 숫자(Digits) 데이터셋을 활용하여 데이터를 분석하고, 다양한 머신러닝 모델을 통해 숫자를 분류하는 과정을 담고 있습니다.

## 데이터셋 소개: Optical Recognition of Handwritten Digits Data
- **데이터 구성**: 총 1,797개의 샘플
- **특성(Features)**: 8x8 픽셀 이미지 (총 64개의 각 픽셀 밝기 값, 0~16 사이의 정수)
- **타겟(Target)**: 0부터 9까지의 숫자 (클래스 10개)
- **목표**: 64개의 픽셀 값을 입력받아 해당 이미지가 어떤 숫자인지 예측하는 분류 모델 구축

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
digits = load_digits()
X = digits.data
y = digits.target

print(f"데이터 크기: {X.shape}")
print(f"타겟 클래스: {np.unique(y)}")

## 1. 탐색적 데이터 분석 (EDA)

In [ ]:
# 데이터프레임 변환 후 기초 통계 확인
df = pd.DataFrame(X, columns=[f"pixel_{i}" for i in range(64)])
df['target'] = y

display(df.head())
print(df.info())

In [ ]:
# 클래스별 분포 시각화
plt.figure(figsize=(8, 4))
sns.countplot(x='target', data=df, palette='viridis')
plt.title('숫자별 데이터 분포')
plt.show()

print("클래스별 샘플 수:")
print(df['target'].value_counts().sort_index())

In [ ]:
# 실제 이미지 시각화
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(digits.images[i], cmap='gray')
    plt.title(f"Label: {y[i]}")
    plt.axis('off')
plt.suptitle('Digits 데이터셋 실제 이미지 예시', fontsize=16)
plt.tight_layout()
plt.show()

## 2. 데이터 전처리 및 특성 엔지니어링

In [ ]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 스케일링 (픽셀값 0~16을 표준화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

## 3. 모델링 (6가지 기초 모델 사용)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# 모델 정의
models = {
    'LogisticRegression': LogisticRegression(max_iter=10000, random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)
}

# 교차 검증을 통한 성능 비교
results = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    results[name] = cv_scores.mean()
    print(f"{name} CV Accuracy: {cv_scores.mean():.4f}")

# 성능 기준 내림차순 정렬
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("\n--- 모델 성능 순위 ---")
for i, (name, score) in enumerate(sorted_results):
    print(f"{i+1}. {name}: {score:.4f}")

## 4. 하이퍼 파라미터 튜닝 및 성능 상위 4개 모델 선정

In [ ]:
# 상위 4개 모델 선정
top_4_model_names = [name for name, score in sorted_results[:4]]
print(f"선정된 상위 4개 모델: {top_4_model_names}")

best_estimators = {}

# SVM 튜닝
if 'SVM' in top_4_model_names:
    param_svm = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 0.01]}
    grid_svm = GridSearchCV(SVC(probability=True, random_state=42), param_svm, cv=3)
    grid_svm.fit(X_train_scaled, y_train)
    best_estimators['SVM'] = grid_svm.best_estimator_

# RandomForest 튜닝
if 'RandomForest' in top_4_model_names:
    param_rf = {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}
    grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_rf, cv=3)
    grid_rf.fit(X_train_scaled, y_train)
    best_estimators['RandomForest'] = grid_rf.best_estimator_

# KNN 튜닝
if 'KNN' in top_4_model_names:
    param_knn = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
    grid_knn = GridSearchCV(KNeighborsClassifier(), param_knn, cv=3)
    grid_knn.fit(X_train_scaled, y_train)
    best_estimators['KNN'] = grid_knn.best_estimator_

# LogisticRegression 튜닝
if 'LogisticRegression' in top_4_model_names:
    param_lr = {'C': [0.1, 1, 10]}
    grid_lr = GridSearchCV(LogisticRegression(max_iter=10000, random_state=42), param_lr, cv=3)
    grid_lr.fit(X_train_scaled, y_train)
    best_estimators['LogisticRegression'] = grid_lr.best_estimator_

# GradientBoosting 튜닝 (필요한 경우)
if 'GradientBoosting' in top_4_model_names:
    param_gb = {'n_estimators': [50, 100], 'learning_rate': [0.05, 0.1]}
    grid_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), param_gb, cv=3)
    grid_gb.fit(X_train_scaled, y_train)
    best_estimators['GradientBoosting'] = grid_gb.best_estimator_

print("튜닝 완료!")

## 5. 앙상블 모델 구축 (Soft Voting)

In [ ]:
from sklearn.ensemble import VotingClassifier

# 튜닝된 상위 4개 모델로 앙상블 구성
estimators = [(name, clf) for name, clf in best_estimators.items()]
voting_clf = VotingClassifier(estimators=estimators, voting='soft')

# 앙상블 모델 학습
voting_clf.fit(X_train_scaled, y_train)

print(f"앙상블 포함 모델: {[name for name, _ in estimators]}")

## 6. 최종 모델 평가

In [ ]:
# 테스트 데이터에 대해 예측
y_pred = voting_clf.predict(X_test_scaled)

print("--- 최종 앙상블 모델 평가 ---")
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Ensemble Model')
plt.show()